In [1]:
!pip install fpdf
import datetime
from fpdf import FPDF
from bs4 import BeautifulSoup
import requests
import re
# scrape standard ke with Python
class SCRAPESTANDARD:
  
  web_url = None   #assign the web_url through a class method
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62'}

  def __init__(self):
    self.get_article = requests.get(SCRAPESTANDARD.web_url,headers=SCRAPESTANDARD.headers)
    self.soup = BeautifulSoup(self.get_article.content,'lxml')

  def find_paragraphs(self): #get paragraphs
    paragraphs = []
    article = self.soup.find('p',class_="card-text").find_next_siblings('p')
    for paragraph in article:
      if paragraph.get_text().strip() != '\n' or paragraph.get_text().strip() != ' ':
        paragraphs.append(paragraph.get_text().strip())
      # return entire article
    par = '\n'.join(paragraphs)
    return par

  def next_prev(self):   #get previous and the next link of a standard article
    next_prev_link = self.soup.find_all('h5',class_='politic-cat')
    prev_link = next_prev_link[0].find_parent()['href']
    next_link = next_prev_link[1].find_parent()['href']
    return next_link,prev_link

  def get_date_timedelta(self):        #get time difference between the date of posting and current date
    # return the date and poster
    # returns also timedifference since the article was posted
    try:
      if self.soup.find('small', class_="text-muted d-none d-md-block") != None:
        date = self.soup.find('small', class_="text-muted d-none d-md-block").get_text().strip()
        split_ = date.split(' ')
        datetime_object = '-'.join([split_[4],list(split_[5])[0],split_[6]])
        date_time = datetime.datetime.strptime(datetime_object,'%B-%d-%Y')
        today = datetime.datetime.now()
        timedelta = today-date_time
        return date,timedelta.days
      else:
        return None,None
    except Exception:
      return None,None

  def get_title(self):               #return the title of the standard article
    try:
      card_title = self.soup.find('h1',class_='card-title')
      return card_title.b.get_text()
    except Exception:
      return 'No Title'

  def topic_title(self):               # return the article category
    links = []
    topics = []
    x = self.soup.find('div',class_="collapse navbar-collapse flex-column ")
    for link in x.find_all('li'):
      if 'href' in link.a.attrs:
        if '#' not in link.a['href'] and link.a['href'] != 'https://www.standardmedia.co.ke':
            topics.append(link.a.get_text())
            links.append(link.a['href'])
    return dict(zip(topics,links))
  
  @classmethod                       #assign the object
  def parse_url(cls,site_url):
    SCRAPESTANDARD.web_url = site_url

In [2]:
!pip install requests
!pip install lxml

In [3]:
class SCRAPENATION:
    num_articles = None
    web_url = None
    delta = None
    http = 'https://nation.africa'
    
    def __init__(self):
        self.get_article = requests.get(SCRAPENATION.web_url,headers=SCRAPESTANDARD.headers)
        self.soup = BeautifulSoup(self.get_article.content,'html.parser')
        
    def all_page_links(self):
        page_links = []
        try:
            links = self.soup.find_all('a')
            for link in links:
                if 'href' in link.attrs:
                    link_ = link['href']
                    pat = re.compile(r'\D\-\d{7}$')
                if link_ not in page_links and re.search(pat,link_):
                    if 'https' in link_:
                        page_links.append(link_)
                    else:
                        page_links.append(SCRAPENATION.http + link_)
        except:
            return None
        return page_links
    
    def get_paragraphs(self):
        pars = []
        try:
            if self.soup != None:
                text_block = self.soup.find_all('div',class_="paragraph-wrapper")
                if text_block == []:
                    text_block = self.soup.find('article', class_="article").find_all('p')
            for par in text_block:
                try:
                    pars.append(par.p.get_text().strip())
                except AttributeError:
                    pars.append(par.get_text().strip())
                except:
                    pass
        except:
            try:
                paragraphs = []
                article = self.soup.find('p',class_="card-text").find_next_siblings('p')
                for paragraph in article:
                  if paragraph.get_text().strip() != '\n' or paragraph.get_text().strip() != ' ':
                    pars.append(paragraph.get_text().strip())
            except AttributeError:
                return None
      # return entire article
    
        return '\n'.join(pars)
    
    def get_author_time(self):
        try:
            time = self.soup.find('time').get_text().strip()
            author = self.soup.find('div','article-authors_texts').p.a.get_text().strip()
        except AttributeError:
            try:
                author = self.soup.find('article', class_="article").find('strong').get_text()
                time = self.soup.find('header').find('h6').get_text()
            except AttributeError:
                return None,None
        return time,author
    
    def get_country_cat(self):
        web_add = SCRAPENATION.web_url.lower().split('/')
        country = web_add[3]
        try:
            if 'historical-flashback' in web_add:
                cat = 'historical-flashback'
            elif 'east-africa' in web_add:
                cat = 'east-africa'
            elif 'africa' in web_add:
                cat = 'africa'
            elif 'counties' in web_add:
                cat = 'counties'
            elif 'news' in web_add:
                cat = 'news'
            elif 'sports' in web_add:
                cat = 'sports'
            elif 'blogs-opinion' in web_add:
                cat = 'blogs-opinion'
            else:
                cat = web_add[4]
            return country.capitalize(),cat.capitalize()
        except:
            return country.capitalize(),None

    def get_title(self):
        try:
            title = self.soup.find('header',class_="article-header").find('h1').get_text().strip()
        except AttributeError:
            title = self.soup.find('title').get_text().strip()
        except Exception:
            title = self.soup.find('header').find('h2').get_text()
        except:
            return None
        return title
    
    def get_article_links(self):
        pat = re.compile(r'\D\-\d{7}$')
        pat1 = re.compile(r'=https')
        pat3 = re.compile(r'/audio/')
        article_links = []
#         try:
        nation_links = SCRAPENATION().all_page_links()
        for link in nation_links:
            SCRAPENATION.parse_url_num_articles_delta(link,SCRAPENATION.num_articles,SCRAPENATION.delta)
            nation = SCRAPENATION()
            time_delta = nation.get_time_delta()
            links = nation.all_page_links()
            nation_links = nation_links+links
            if link not in article_links and time_delta <= SCRAPENATION.delta:
                if re.search(pat,link):
                    if not re.search(pat1,link) and not re.search(pat3,link):
                        article_links.append(link)
                        nation_links.append(link)
                        print(link)
                else:
                    nation_links.append(link)
            if len(article_links) > SCRAPENATION.num_articles:
                break
        return article_links
#         except:
#             return article_links

    def get_time_delta(self):
        time,author =  SCRAPENATION().get_author_time()
        if time == None:
            return 0
        else:
            try:
                date_time = datetime.datetime.strptime(time,'%A, %B %d, %Y')
                today = datetime.datetime.now()
                time_delta = (today - date_time).days
                return time_delta
            except ValueError:
                try:
                    date_time = datetime.datetime.strptime(time,'%A %B %d %Y')
                    today = datetime.datetime.now()
                    time_delta = (today - date_time).days
                    return time_delta
                except ValueError:
                    return 5
#     @staticmethod

    def get_articles(self):
        news = []
        history = []
        articles = []
        counties = []
        africa = []
        east_africa = []
        sports = []
        opinion = []
        business = []
#         try:
        articlelinks = SCRAPENATION().get_article_links()
        for article in articlelinks:
            SCRAPENATION.parse_url_num_articles_delta(article,SCRAPENATION.num_articles,SCRAPENATION.delta)
            nation = SCRAPENATION()
            title = nation.get_title()
            par = nation.get_paragraphs()
            country,cat = nation.get_country_cat()
            time,author =  nation.get_author_time()
            cate= cat.lower()
            if par != None and cate != 'audio':
                if len(par) > 500:
                    if time == None:
                        time = 'UnKnown Date'
                    if author == None:
                        author = 'Unknown Author'
                    if title == None:
                        title = 'Untitled'
                    if cate== 'historical-flashback':
                        history.append([title,cat,author,time,par])
                    elif cate== 'east-africa':
                        east_africa.append([title,cat,author,time,par])
                    elif cate== 'africa':
                        africa.append([title,cat,author,time,par])
                    elif cate== 'counties':
                        counties.append([title,cat,author,time,par])
                    elif cate== 'news':
                        news.append([title,cat,author,time,par])
                    elif cate== 'sports':
                        sports.append([title,cat,author,time,par])
                    elif cate== 'blogs-opinion':
                        opinion.append([title,cat,author,time,par])
                    elif cate == 'business':
                        business.append([title,cat,author,time,par])
                    else:
                        articles.append([title,cat,author,time,par])
        return news+counties+opinion+east_africa+africa+history+sports+articles
#         except:
#             return news+counties+opinion+east_africa+africa+history+sports+articles

    @classmethod
    def parse_url_num_articles_delta(cls,url,num,time_delta):
        cls.web_url = url
        cls.num_articles = num
        cls.delta= time_delta
        
        
        


In [4]:
web = 'https://nation.africa/kenya'
SCRAPENATION.parse_url_num_articles_delta(web,1000,1)
nation = SCRAPENATION()

In [5]:
par = nation.get_paragraphs()
country,cat = nation.get_country_cat()
time,author =  nation.get_author_time()
time_delta = nation.get_time_delta()
title = nation.get_title()
links = nation.all_page_links()
# articles = nation.get_article_links()

In [6]:
articlez = nation.get_articles()

https://nation.africa/kenya/news/bbi-is-unconstitutional-court-of-appeal-rules-3519062
https://nation.africa/kenya/news/bbi-read-the-full-appellate-court-judgment--3519292
https://nation.africa/kenya/news/bbi-court-of-appeal-judges-give-verdict-3518090
https://nation.africa/kenya/sports/athletics/jackline-chepkoech-glides-to-3000m-steeplechase-gold-3518970
https://nation.africa/kenya/news/politics/the-bbi-journey-from-start-to-finish--3518928
https://nation.africa/kenya/news/africa/uganda-suspends-54-ngos-including-prominent-rights-group-3518458
https://nation.africa/kenya/counties/embu/embu-brothers-died-the-day-they-started-a-butchery-business--3518578
https://nation.africa/kenya/news/mp-alleges-bias-in-quest-for-chinese-visas-3518330
https://nation.africa/kenya/news/world/taliban-revenge-fears-grow-in-afghanistan-3518996
https://nation.africa/kenya/business/technology/elon-musk-says-tesla-s-robot-will-make-physical-work-a-choice--3518366
https://nation.africa/kenya/news/covid-19-ken

In [7]:
for article in articlez:
    title,category,author,date,article = article
    author_date = author+' | ' +  date
    print(category,author_date)

News Joseph Wangui | Saturday, August 21, 2021
News Unknown Author | UnKnown Date
News Unknown Author | UnKnown Date
News Silas Apollo | Friday, August 20, 2021
News Samwel Owino | Friday, August 20, 2021
News AFP | Friday, August 20, 2021
News Amina Wako | Friday, August 20, 2021
News Anthony Kitimo | Friday, August 20, 2021
Counties George Munene | Friday, August 20, 2021
Counties Steve Njuguna | Saturday, August 21, 2021
Counties Brian Ocharo | Friday, August 20, 2021
Counties Philip Muyanga | Friday, August 20, 2021
Counties Siago Cece | Friday, August 20, 2021
Counties Mercy Mwende | Friday, August 20, 2021
Counties Brian Ocharo | Friday, August 20, 2021
Counties Lucy Mkanyika | Friday, August 20, 2021
Counties Maureen Ongala | Friday, August 20, 2021
Blogs-opinion Magesha Ngwiri | Saturday, August 21, 2021
Blogs-opinion Eric Ng'eno | Saturday, August 21, 2021
Blogs-opinion Gabriel Oguda | Saturday, August 21, 2021
Blogs-opinion Njoki Chege | Saturday, August 21, 2021
Blogs-opinio

In [8]:
# cast the scraped articles into a pdf

# modify the footer and the head of the class
class PDF(FPDF):
    topic = 'none'
    title = ''
    date = 'none'
    def header(self):
        # add topic_title and logo
        self.image(r'nation.png',12,10,25)
        self.set_text_color(0,0,0)
        self.set_font('helvetica','BI',15)
        self.cell(200,10,PDF.topic.encode('latin-1', 'ignore').decode('latin-1'),border=0,align='C')
        self.ln()
        # set font
        self.set_font('helvetica','B',14)
        doc_w = self.w
        length_title = self.get_string_width(title)+6
        self.set_x((doc_w-length_title)/2)
        self.set_draw_color(0,80,100)
        self.set_text_color(220,50,50)
        self.set_line_width(1)
        # set the title of the page
        self.cell(length_title,10,PDF.title.encode('latin-1', 'ignore').decode('latin-1'),border = False,ln=1,align='C')
        # break the line
        self.ln()
        self.set_font('times','I',12)
        self.set_text_color(0,128,128)
        self.cell(0,5,PDF.date.encode('latin-1', 'ignore').decode('latin-1'))
        self.ln()
    def footer(self):
        self.set_y(-10)
        self.set_text_color(128,128,0)
        self.set_font('helvetica','B',10)
        self.cell(0,10,f'{self.page_no()}',align='C')
    def chapter_body(self,text):
        for line in text.splitlines():
            line_ =line.strip()
            txt = line_.encode('latin-1', 'ignore').decode('latin-1')
            self.set_font('times','',11)
            self.multi_cell(0,5,txt)
            self.ln(h=0.7)
            self.set_font('times','I',12)
        self.cell(0,5,'End of'+' '+ title.encode('latin-1', 'ignore').decode('latin-1'))
    @classmethod
    def title_topic_date(cls,title,topic,date):
        PDF.title = title.encode('latin-1', 'ignore').decode('latin-1')
        PDF.topic = topic.encode('latin-1', 'ignore').decode('latin-1')
        PDF.date = date.encode('latin-1', 'ignore').decode('latin-1')

In [9]:
len(articlez)

43

In [10]:
# print the pdf copy of the scraped articles
pdf =PDF()
n= 0
for article in articlez:
    title,category,author,date,article = article
    author_date = author+' | ' +  date
    PDF.title_topic_date(title,category,author_date)
    pdf.add_page()
    pdf.set_auto_page_break(auto=True,margin=10)
    pdf.chapter_body(article)
    n = n+1

day = datetime.datetime.today().day
pdf.output(f'pdf{day}.pdf')

''